リスト13.1 プレーンブロックチェーン

In [1]:
import hashlib
import datetime
import time
import json

INITIAL_BITS = 0x1e777777
MAX_32BIT = 0xffffffff

class Block():
    def __init__(self, index, prev_hash, data, timestamp, bits):
        self.index = index
        self.prev_hash = prev_hash
        self.data = data
        self.timestamp = timestamp
        self.bits = bits
        self.nonce = 0
        self.elapsed_time = ""
        self.block_hash = ""

    def __setitem__(self, key, value):
        setattr(self, key, value)

    def to_json(self):
        return {
            "index"       : self.index,
            "prev_hash"   : self.prev_hash,
            "stored_data" : self.data,
            "timestamp"   : self.timestamp.strftime("%Y/%m/%d %H:%M:%S"),
            "bits"        : hex(self.bits)[2:].rjust(8, "0"),
            "nonce"       : hex(self.nonce)[2:].rjust(8, "0"),
            "elapsed_time": self.elapsed_time,
            "block_hash"  : self.block_hash
        }

    def calc_blockhash(self):
        blockheader = str(self.index) + str(self.prev_hash) + str(self.data) + str(self.timestamp) + hex(self.bits)[2:] + str(self.nonce)
        h = hashlib.sha256(blockheader.encode()).hexdigest()
        self.block_hash = h
        return h
    
    def calc_target(self):
        exponent_bytes = (self.bits >> 24) - 3
        exponent_bits = exponent_bytes * 8
        coefficient = self.bits & 0xffffff
        return coefficient << exponent_bits
    
    def check_valid_hash(self):
        return int(self.calc_blockhash(), 16) <= self.calc_target()

class Blockchain():
    def __init__(self, initial_bits):
        self.chain = []
        self.initial_bits = initial_bits

    def add_block(self, block):
        self.chain.append(block)
    
    def getblockinfo(self, index=-1):
        return print(json.dumps(self.chain[index].to_json(), indent=2, sort_keys=True, ensure_ascii=False))
    
    def mining(self, block):
        start_time = int(time.time() * 1000)
        while True:
            for n in range(MAX_32BIT + 1):
                block.nonce = n
                if block.check_valid_hash():
                    end_time = int(time.time() * 1000)
                    block.elapsed_time = \
                    str((end_time - start_time) / \
                        1000.0) + "秒"
                    self.add_block(block)
                    self.getblockinfo()
                    return
            new_time = datetime.datetime.now()
            if new_time == block.timestamp:
                block.timestamp += datetime.timedelta(seconds=1)
            else:
                block.timestamp = new_time
    
    def create_genesis(self):
        genesis_block = Block(0, "0000000000000000000000000000000000000000000000000000000000000000", "ジェネシスブロック", datetime.datetime.now(), self.initial_bits)
        self.mining(genesis_block)
    
    def add_newblock(self, i):
        last_block = self.chain[-1]
        block = Block(i+1, last_block.block_hash, "ブロック " + str(i+1), datetime.datetime.now(), last_block.bits)
        self.mining(block)

if __name__ == "__main__":
    bc = Blockchain(INITIAL_BITS)
    print("ジェネシスブロックを作成中・・・")
    bc.create_genesis()
    for i in range(30):
        print(str(i+2) + "番目のブロックを作成中・・・")
        bc.add_newblock(i)

ジェネシスブロックを作成中・・・
{
  "bits": "1e777777",
  "block_hash": "00004e04b324728d6eab2c0dd09298f3294f3837f45899f429934e0de5aba09e",
  "elapsed_time": "0.253秒",
  "index": 0,
  "nonce": "0000fdfe",
  "prev_hash": "0000000000000000000000000000000000000000000000000000000000000000",
  "stored_data": "ジェネシスブロック",
  "timestamp": "2022/08/28 12:30:24"
}
2番目のブロックを作成中・・・
{
  "bits": "1e777777",
  "block_hash": "00004f8bfb768104df500c0a34da3eabaf8f82976f1ae5224a1b2343763df559",
  "elapsed_time": "0.23秒",
  "index": 1,
  "nonce": "0000eacb",
  "prev_hash": "00004e04b324728d6eab2c0dd09298f3294f3837f45899f429934e0de5aba09e",
  "stored_data": "ブロック 1",
  "timestamp": "2022/08/28 12:30:25"
}
3番目のブロックを作成中・・・
{
  "bits": "1e777777",
  "block_hash": "00006da41d9c98a9732d91b20f346d3ac1b9c8f3196de0f0b765e7c402c0b0ba",
  "elapsed_time": "0.317秒",
  "index": 2,
  "nonce": "000149da",
  "prev_hash": "00004f8bfb768104df500c0a34da3eabaf8f82976f1ae5224a1b2343763df559",
  "stored_data": "ブロック 2",
  "timestamp": "2022/0

{
  "bits": "1e777777",
  "block_hash": "00003c7ac90f31639ec1a091fafae05f43734ee85fea2222ebb4facc8bbb9269",
  "elapsed_time": "0.959秒",
  "index": 25,
  "nonce": "0003c626",
  "prev_hash": "00005c6cee618aa2dc4418b69e5f1518e2ade7179f1e3f74db73484099670533",
  "stored_data": "ブロック 25",
  "timestamp": "2022/08/28 12:30:39"
}
27番目のブロックを作成中・・・
{
  "bits": "1e777777",
  "block_hash": "0000519d84e947684c0b65ce50702640381c62fa56463f81b1c0837bf2b53dff",
  "elapsed_time": "0.282秒",
  "index": 26,
  "nonce": "00011951",
  "prev_hash": "00003c7ac90f31639ec1a091fafae05f43734ee85fea2222ebb4facc8bbb9269",
  "stored_data": "ブロック 26",
  "timestamp": "2022/08/28 12:30:40"
}
28番目のブロックを作成中・・・
{
  "bits": "1e777777",
  "block_hash": "00006ac3757a7dc2565f7b53da2e4c0aa12d5b5a51cfd0d0d2ae040715bfb151",
  "elapsed_time": "0.376秒",
  "index": 27,
  "nonce": "000170ab",
  "prev_hash": "0000519d84e947684c0b65ce50702640381c62fa56463f81b1c0837bf2b53dff",
  "stored_data": "ブロック 27",
  "timestamp": "2022/08/28 12:30:

リスト13.2 リスト13.1 の出力結果（JSON形式）、（リスト13.2はリスト13.1の出力結果をご覧ください）

リスト13.3　リスト13.1 のINITIAL_BITSの値を、"1e777777"から"1d777777"に変更した出力結果（JSON形式）

In [4]:
import hashlib
import datetime
import time
import json

INITIAL_BITS = 0x1d777777
MAX_32BIT = 0xffffffff

class Block():
    def __init__(self, index, prev_hash, data, timestamp, bits):
        self.index = index
        self.prev_hash = prev_hash
        self.data = data
        self.timestamp = timestamp
        self.bits = bits
        self.nonce = 0
        self.elapsed_time = ""
        self.block_hash = ""

    def __setitem__(self, key, value):
        setattr(self, key, value)

    def to_json(self):
        return {
            "index"       : self.index,
            "prev_hash"   : self.prev_hash,
            "stored_data" : self.data,
            "timestamp"   : self.timestamp.strftime("%Y/%m/%d %H:%M:%S"),
            "bits"        : hex(self.bits)[2:].rjust(8, "0"),
            "nonce"       : hex(self.nonce)[2:].rjust(8, "0"),
            "elapsed_time": self.elapsed_time,
            "block_hash"  : self.block_hash
        }

    def calc_blockhash(self):
        blockheader = str(self.index) + str(self.prev_hash) + str(self.data) + str(self.timestamp) + hex(self.bits)[2:] + str(self.nonce)
        h = hashlib.sha256(blockheader.encode()).hexdigest()
        self.block_hash = h
        return h
    
    def calc_target(self):
        exponent_bytes = (self.bits >> 24) - 3
        exponent_bits = exponent_bytes * 8
        coefficient = self.bits & 0xffffff
        return coefficient << exponent_bits
    
    def check_valid_hash(self):
        return int(self.calc_blockhash(), 16) <= self.calc_target()

class Blockchain():
    def __init__(self, initial_bits):
        self.chain = []
        self.initial_bits = initial_bits

    def add_block(self, block):
        self.chain.append(block)
    
    def getblockinfo(self, index=-1):
        return print(json.dumps(self.chain[index].to_json(), indent=2, sort_keys=True, ensure_ascii=False))
    
    def mining(self, block):
        start_time = int(time.time() * 1000)
        while True:
            for n in range(MAX_32BIT + 1):
                block.nonce = n
                if block.check_valid_hash():
                    end_time = int(time.time() * 1000)
                    block.elapsed_time = str((end_time - start_time) / 1000.0) + "秒"
                    self.add_block(block)
                    self.getblockinfo()
                    return
            new_time = datetime.datetime.now()
            if new_time == block.timestamp:
                block.timestamp += datetime.timedelta(seconds=1)
            else:
                block.timestamp = new_time
    
    def create_genesis(self):
        genesis_block = Block(0, "0000000000000000000000000000000000000000000000000000000000000000", "ジェネシスブロック", datetime.datetime.now(), self.initial_bits)
        self.mining(genesis_block)
    
    def add_newblock(self, i):
        last_block = self.chain[-1]
        block = Block(i+1, last_block.block_hash, "ブロック " + str(i+1), datetime.datetime.now(), last_block.bits)
        self.mining(block)

if __name__ == "__main__":
    bc = Blockchain(INITIAL_BITS)
    print("ジェネシスブロックを作成中・・・")
    bc.create_genesis()
    for i in range(30):
        print(str(i+2) + "番目のブロックを作成中・・・")
        bc.add_newblock(i)


ジェネシスブロックを作成中・・・
{
  "bits": "1d777777",
  "block_hash": "0000000dc763c605cc883b2e5ddc776b8c100be4971de61b522fd9edce595e03",
  "elapsed_time": "93.604 秒",
  "index": 0,
  "nonce": "016cb281",
  "prev_hash": "0000000000000000000000000000000000000000000000000000000000000000",
  "stored_data": "ジェネシスブロック",
  "timestamp": "2019/10/02 18:39:11"
}
2番目のブロックを作成中・・・
{
  "bits": "1d777777",
  "block_hash": "0000001b67409cbe40d13edb188772949e33b038014baba694c6b2ce73955761",
  "elapsed_time": "209.994 秒",
  "index": 1,
  "nonce": "03432c25",
  "prev_hash": "0000000dc763c605cc883b2e5ddc776b8c100be4971de61b522fd9edce595e03",
  "stored_data": "ブロック 1",
  "timestamp": "2019/10/02 18:40:44"
}
3番目のブロックを作成中・・・
{
  "bits": "1d777777",
  "block_hash": "000000755bee0428a2f7c565bb366ee43472466106dadc3fe95c5385b11b240c",
  "elapsed_time": "29.004 秒",
  "index": 2,
  "nonce": "00737f07",
  "prev_hash": "0000001b67409cbe40d13edb188772949e33b038014baba694c6b2ce73955761",
  "stored_data": "ブロック 2",
  "timestamp":

{
  "bits": "1d777777",
  "block_hash": "00000020ed589d900b80763d48ba5eb2f0cd1a393f04f758b256116785273756",
  "elapsed_time": "228.242 秒",
  "index": 24,
  "nonce": "0386b9e2",
  "prev_hash": "00000064d8cef7d02f0b7168b11afe8dd6ee40a03100e8cc8961c93e80e23c59",
  "stored_data": "ブロック 24",
  "timestamp": "2019/10/02 19:38:10"
}
26番目のブロックを作成中・・・
{
  "bits": "1d777777",
  "block_hash": "000000023704c965f304a38c492be312450437363172629cdebe5196856f90b8",
  "elapsed_time": "48.153 秒",
  "index": 25,
  "nonce": "00bde943",
  "prev_hash": "00000020ed589d900b80763d48ba5eb2f0cd1a393f04f758b256116785273756",
  "stored_data": "ブロック 25",
  "timestamp": "2019/10/02 19:41:58"
}
27番目のブロックを作成中・・・
{
  "bits": "1d777777",
  "block_hash": "0000001cd952eaff827f84a322e52164d2e7436555bdc793d327d0adf6bc4bb5",
  "elapsed_time": "108.54 秒",
  "index": 26,
  "nonce": "01abe0d8",
  "prev_hash": "000000023704c965f304a38c492be312450437363172629cdebe5196856f90b8",
  "stored_data": "ブロック 26",
  "timestamp": "2019/10/02

リスト13.4 Blockクラスの変数

In [ ]:
def __init__(self, index, prev_hash, data, timestamp, bits):
    self.index = index
    self.prev_hash = prev_hash
    self.data = data
    self.timestamp = timestamp
    self.bits = bits
    self.nonce = 0
    self.elapsed_time = ""
    self.block_hash = ""

リスト13.5 特殊メソッドsetitem

In [ ]:
def __setitem__(self, key, value):
    setattr(self, key, value)

リスト13.6 to_jsonメソッド

In [ ]:
def to_json(self):
return {
            "index"       : self.index,
            "prev_hash"   : self.prev_hash,
            "stored_data" : self.data,
            "timestamp"   : self.timestamp.strftime("%Y/%m/%d %H:%M:%S"),
            "bits"        : hex(self.bits)[2:].rjust(8, "0"),
            "nonce"       : hex(self.nonce)[2:].rjust(8, "0"),
            "elapsed_time": self.elapsed_time,
            "block_hash"  : self.block_hash
        }

リスト13.7 calc_blockhashメソッド

In [ ]:
def calc_blockhash(self):
    blockheader = str(self.index) + str(self.prev_hash) + str(self.data) + str(self.timestamp) + hex(self.bits)[2:] + str(self.nonce)
    h = hashlib.sha256(blockheader.encode()).hexdigest()
    self.block_hash = h
    return h

リスト13.8 calc_targetメソッド

In [ ]:
def calc_target(self):
    exponent_bytes = (self.bits >> 24) - 3
    exponent_bits = exponent_bytes * 8
    coefficient = self.bits & 0xffffff
    return coefficient << exponent_bits

リスト13.9 check_valid_hashメソッド

In [ ]:
def check_valid_hash(self):
    return int(self.calc_blockhash(), 16) <= self.calc_target()

リスト13.10 Blockchainクラスの変数

In [ ]:
def __init__(self, initial_bits):
    self.chain = []
    self.initial_bits = initial_bits

リスト13.11 add_blockメソッド

In [ ]:
def add_block(self, block):
    self.chain.append(block)

リスト13.12 getblockinfoメソッド

In [ ]:
def getblockinfo(self, index=-1):
    return print(json.dumps(self.chain[index].to_json(), indent=2, sort_keys=True, ensure_ascii=False))


リスト13.13 miningメソッド

In [ ]:
def mining(self, block):
    start_time = int(time.time() * 1000)
    while True:
        for n in range(MAX_32BIT + 1):
            block.nonce = n
            if block.check_valid_hash():
                end_time = int(time.time() * 1000)
                block.elapsed_time = \
                str((end_time - start_time) / \
                    1000.0) + "秒"
                self.add_block(block)
                self.getblockinfo()
                return
        new_time = datetime.datetime.now()
        if new_time == block.timestamp:
            block.timestamp += datetime.timedelta(seconds=1)
        else:
            block.timestamp = new_time

リスト13.14 create_genesisメソッド

In [ ]:
def create_genesis(self):
    genesis_block = Block(0, "0000000000000000000000000000000000000000000000000000000000000000", "ジェネシスブロック", datetime.datetime.now(), self.initial_bits)
    self.mining(genesis_block) 


リスト13.15 add_newblock関数（plain-blockchain.py）

In [ ]:
def add_newblock(self, i):
    last_block = self.chain[-1]
    block = Block(i+1, last_block.block_hash, "ブロック " + str(i+1), datetime.datetime.now(), last_block.bits)
    self.mining(block)

リスト13.16  Blockchainクラスのインスタンス化

In [ ]:
if __name__ == "__main__":
    bc = Blockchain(INITIAL_BITS)
    print("ジェネシスブロックを作成中・・・")
    bc.create_genesis()
    for i in range(30):
        print(str(i+2) + "番目のブロックを作成中・・・")
        bc.add_newblock(i) 